In [7]:
import functools
import tensorflow as tf

In [8]:
def lazy_property(function):
    attribute = '_' + function.__name__

    @property
    @functools.wraps(function)
    def wrapper(self):
        if not hasattr(self, attribute):
            setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return wrapper

In [17]:
class VariableSequenceClassification:

    def __init__(self, data, target, num_hidden=200, num_layers=2):
        self.data = data
        self.target = target
        self._num_hidden = num_hidden
        self._num_layers = num_layers
        self.prediction
        self.error
        self.optimize

    @lazy_property
    def length(self):
        used = tf.sign(tf.reduce_max(tf.abs(self.data), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length

    @lazy_property
    def prediction(self):
        # Recurrent network.
        output, _ = tf.nn.dynamic_rnn(
            tf.nn.rnn_cell.GRUCell(self._num_hidden),
            data,
            dtype=tf.float32,
            sequence_length=self.length,
        )
        last = self._last_relevant(output, self.length)
        # Softmax layer.
        weight, bias = self._weight_and_bias(
            self._num_hidden, int(self.target.get_shape()[1]))
        prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
        return prediction

    @lazy_property
    def cost(self):
        cross_entropy = -tf.reduce_sum(self.target * tf.log(self.prediction))
        return cross_entropy

    @lazy_property
    def optimize(self):
        learning_rate = 0.003
        optimizer = tf.train.RMSPropOptimizer(learning_rate)
        return optimizer.minimize(self.cost)

    @lazy_property
    def error(self):
        mistakes = tf.not_equal(
            tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
        return tf.reduce_mean(tf.cast(mistakes, tf.float32))

    @staticmethod
    def _weight_and_bias(in_size, out_size):
        weight = tf.truncated_normal([in_size, out_size], stddev=0.01)
        bias = tf.constant(0.1, shape=[out_size])
        return tf.Variable(weight), tf.Variable(bias)

    @staticmethod
    def _last_relevant(output, length):
        batch_size = tf.shape(output)[0]
        max_length = int(output.get_shape()[1])
        output_size = int(output.get_shape()[2])
        index = tf.range(0, batch_size) * max_length + (length - 1)
        flat = tf.reshape(output, [-1, output_size])
        relevant = tf.gather(flat, index)
        return relevant


In [24]:
import tensorflow as tf
import numpy as np
from toy_seq_data import ToySequenceData
from general_tools.notebook.tf import reset_tf_graph

In [25]:
reset_tf_graph()
max_steps = 20
n_classes = 2
n_features = 1

trainset = ToySequenceData(n_samples=1000, max_seq_len=max_steps)
testset = ToySequenceData(n_samples=500, max_seq_len=max_steps)


data = tf.placeholder(tf.float32, [None, max_steps, n_features])
target = tf.placeholder(tf.float32, [None, n_classes])

model = VariableSequenceClassification(data, target)

In [29]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
for epoch in range(10):
    for _ in range(100):
        batch = trainset.next(10)
        sess.run(model.optimize, {data: batch[0], target: batch[1]})
        error = sess.run(model.error, {data: batch[0], target: batch[1]})
        print('Epoch {:2d} error {:3.1f}%'.format(epoch + 1, 100 * error))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch  1 error 40.0%
Epoch  1 error 30.0%
Epoch  1 error 30.0%
Epoch  1 error 20.0%
Epoch  1 error 50.0%
Epoch  1 error 60.0%
Epoch  1 error 60.0%
Epoch  1 error 20.0%
Epoch  1 error 60.0%
Epoch  1 error 30.0%
Epoch  1 error 50.0%
Epoch  1 error 70.0%
Epoch  1 error 60.0%
Epoch  1 error 50.0%
Epoch  1 error 40.0%
Epoch  1 error 60.0%
Epoch  1 error 20.0%
Epoch  1 error 40.0%
Epoch  1 error 20.0%
Epoch  1 error 40.0%
Epoch  1 error 50.0%
Epoch  1 error 30.0%
Epoch  1 error 40.0%
Epoch  1 error 40.0%
Epoch  1 error 40.0%
Epoch  1 error 30.0%
Epoch  1 error 30.0%
Epoch  1 error 60.0%
Epoch  1 error 50.0%
Epoch  1 error 30.0%
Epoch  1 error 70.0%
Epoch  1 error 40.0%
Epoch  1 error 60.0%
Epoch  1 error 50.0%
Epoch  1 error 50.0%
Epoch  1 error 60.0%
Epoch  1 error 30.0%
Epoch  1 error 30.0%
Epoch  1 error 30.0%
Epoch  1 error 40.0%
Epoch  1 error 30.0%
Epoch  1 error 40.0%
Epoch  1 error 60.0%
Epoch  1 error 40.0%
Ep

In [43]:
batch = testset.next(10)
error = sess.run(model.error, {data: batch[0], target: batch[1]})
print error 

0.0
